# 뉴럴 네트워크의 구현
이번에 우리는 완전 연결 레이어로 뉴럴 네트워크를 만들어 분류를 수행하고 CIFAR-10 데이터셋으로 테스트 해볼 것 입니다.

In [ ]:
# 설치

import numpy as np
import matplotlib.pyplot as plt

from cs231n.classifiers.neural_net import TwoLayerNet

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # 기본 그래프 사이즈 설정
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# 외부 모듈 자동 불러오기
# 참고. http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

def rel_error(x, y):
  """ returns relative error """
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

우리는 네트워크의 인스턴스를 나타내기 위해 `cs231n/classifiers/neural_net.py` 파일의 `TwoLayerNet` 클래스를 사용할 것입니다. 네트워크 파라메터는 인스턴스 변수 `self.params`에 키는 파라메터 이름인 문자열이고 값은 numpy 배열로 저장되어 있습니다. 아래에서 구현에 사용할 toy 데이터와 toy 모델을 초기화 합니다.

In [ ]:
# 작은 net을 만들고 toy 데이터로 구현을 체크해 봅니다.
# 반복되는 실험에서 우리가 랜덤 시드를 설정한다는 것을 주의하세요.

input_size = 4
hidden_size = 10
num_classes = 3
num_inputs = 5

def init_toy_model():
  np.random.seed(0)
  return TwoLayerNet(input_size, hidden_size, num_classes, std=1e-1)

def init_toy_data():
  np.random.seed(1)
  X = 10 * np.random.randn(num_inputs, input_size)
  y = np.array([0, 1, 2, 2, 1])
  return X, y

net = init_toy_model()
X, y = init_toy_data()

# Forward pass: 점수 계산하기
`cs231n/classifiers/neural_net.py`파일을 열고 `TwoLayerNet.loss` 방법에 대해서 확인해 보세요. 이 함수는 SVM과 Softmax에서 작성했던 손실함수와 매우 유사합니다: 데이터와 가중치로 클래스의 점수, 손실정도, 매개변수의 그라디언트를 계산합니다.

Forward pass의 첫 번째 부분의 구현은 모든 입력에 대한 점수를 계산하기 위해 가중치와 biases를 사용합니다.

In [ ]:
scores = net.loss(X)
print 'Your scores:'
print scores
print
print 'correct scores:'
correct_scores = np.asarray([
  [-0.81233741, -1.27654624, -0.70335995],
  [-0.17129677, -1.18803311, -0.47310444],
  [-0.51590475, -1.01354314, -0.8504215 ],
  [-0.15419291, -0.48629638, -0.52901952],
  [-0.00618733, -0.12435261, -0.15226949]])
print correct_scores
print

# 차이가 매우 작을 것입니다. 우리는 <1e-7 정도 나왔습니다.
print 'Difference between your scores and correct scores:'
print np.sum(np.abs(scores - correct_scores))

# Forward pass: 손실 계산하기
같은 함수에서, 데이터와 정규화 손실 정도를 계산하는 부분을 구현해 봅시다.

In [ ]:
loss, _ = net.loss(X, y, reg=0.1)
correct_loss = 1.30378789133

# 매우 작을 것 입니다, 우리는 1e-12보다 적은 값을 얻었습니다.
print 'Difference between your loss and correct loss:'
print np.sum(np.abs(loss - correct_loss))

# Backward pass
함수의 남은 부분을 구현합니다. W1, b1, W2, b2 변수들에 대한 손실함수의 그라디언트를 구현합니다. 이제 정확한 froward pass를 구현했습니다, 이제 backward pass를 수치 그라디언트 체크로 디버그 할 수 있을 것 입니다.

In [ ]:
from cs231n.gradient_check import eval_numerical_gradient

# 수치 그라디언트 체크로 backward pass에서 구현한 것을 체크합니다.
# 만약 구현이 맞았다면, 각 W1, W2, b1, b2에 대해서 numeric 과 통계적 그라디언트는 1e-8 이하의 차이가 있을 것 입니다.

loss, grads = net.loss(X, y, reg=0.1)

# 모두 해봐야 1e-8 이하 정도일 것입니다.
for param_name in grads:
  f = lambda W: net.loss(X, y, reg=0.1)[0]
  param_grad_num = eval_numerical_gradient(f, net.params[param_name], verbose=False)
  print '%s max relative error: %e' % (param_name, rel_error(param_grad_num, grads[param_name]))

# 네트워크 학습
네트워크를 학습시키기 위해 우리는 SVM과 Softmax분류기와 비슷한 stochastic gradient descent(SGD)를 사용할 것입니다. `TwoLayerNet.train` 함수를 보고 비워있는 부분을 채워 넣어서 학습 프로시저를 구현해 보세요. 이것은 SVM과 Softmax 분류기에서 사용한 학습 과정과 매우 비슷할 것입니다. 또한 학습 과정은 정기적으로 네트워크가 학습되는 동안 정확도를 유지하기위한 예측모델을 수행하는 `TwoLayerNet.predict`도 구현해야 합니다.

한번 메서드를 구현하면, 아래의 코드를 실행시켜 toy 데이터의 two-layer 네트워크를 학습시킵니다. 손실은 0.2미만이어야 합니다.

In [ ]:
net = init_toy_model()
stats = net.train(X, y, X, y,
            learning_rate=1e-1, reg=1e-5,
            num_iters=100, verbose=False)

print 'Final training loss: ', stats['loss_history'][-1]

# 손실 기록 그래프
plt.plot(stats['loss_history'])
plt.xlabel('iteration')
plt.ylabel('training loss')
plt.title('Training Loss history')
plt.show()

# 데이터 불러오기
이제 그라데이션 검사를 통과하고 toy 데이터에서 작동하는 two-layer 네트워크를 구현해야 합니다.
분류기에 실제 데이터셋을 학습시키기위해 우리가 좋아하는 CIFAR-10 데이터를 불러올 시간입니다.

In [ ]:
from cs231n.data_utils import load_CIFAR10

def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000):
    """
    CIFAR-10 데이터셋을 디스크에서 불러와서 two-layer 신경 망 분류기를 위해 준비한 사전 작업을
    수행합니다. 우리가 SVM에서 했던 작업과 비슷하지만 하나의 함수로 축약되어 있습니다.
    """
    # 원본 CIFAR-10 데이터를 불러옵니다.
    cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
        
    # 데이터 표본
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # 데이터를 정규화 시킵니다.: 평균 이미지를 뺍니다.
    mean_image = np.mean(X_train, axis=0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image

    # 데이터를 열(row)로 변형시킵니다.
    X_train = X_train.reshape(num_training, -1)
    X_val = X_val.reshape(num_validation, -1)
    X_test = X_test.reshape(num_test, -1)

    return X_train, y_train, X_val, y_val, X_test, y_test


# 데이터를 얻기위해 위의 함수들을 호출합니다.
X_train, y_train, X_val, y_val, X_test, y_test = get_CIFAR10_data()
print 'Train data shape: ', X_train.shape
print 'Train labels shape: ', y_train.shape
print 'Validation data shape: ', X_val.shape
print 'Validation labels shape: ', y_val.shape
print 'Test data shape: ', X_test.shape
print 'Test labels shape: ', y_test.shape

# 망 학습시키기
네트워크를 학습시키기 위해 모멘텀과 SGD를 사용합니다. 추가적으로 지수적인 학습 정도를 최적화되도로 조절합니다;
각 epoch 후에 학습 정도를 decay rate를 곱해서 감소시킵니다.

In [ ]:
input_size = 32 * 32 * 3
hidden_size = 50
num_classes = 10
net = TwoLayerNet(input_size, hidden_size, num_classes)

# 망 학습시키기
stats = net.train(X_train, y_train, X_val, y_val,
            num_iters=1000, batch_size=200,
            learning_rate=1e-4, learning_rate_decay=0.95,
            reg=0.5, verbose=True)

# 검증 셋에 대해 확인하기
val_acc = (net.predict(X_val) == y_val).mean()
print 'Validation accuracy: ', val_acc



# 디버그
위에서 제공한 기본 파라메터로, 0.29 정도의 검증 정확도를 얻을 수 있을 것입니다. 별로 좋지 않죠.

통찰력을 얻기위한 하나의 전략은 최적화 중의 학습과 검증 셋에 대한 손실 함수와 정확도 그래프가 틀린 이유를 찾아보는 것 입니다.

다른 전략은 네트워크의 첫 레이어가 학습한 가중치를 시각화 해보는 것입니다. 대부분 시각 데이터를 학습한 뉴럴 네트워크의 첫 레이어의 가중치는 일반적으로 시각화 했을때 몇 가지 눈에보이는 구조를 갖습니다.

In [ ]:
# 손실함수와 학습 / 검증 정확도 그래프
plt.subplot(2, 1, 1)
plt.plot(stats['loss_history'])
plt.title('Loss history')
plt.xlabel('Iteration')
plt.ylabel('Loss')

plt.subplot(2, 1, 2)
plt.plot(stats['train_acc_history'], label='train')
plt.plot(stats['val_acc_history'], label='val')
plt.title('Classification accuracy history')
plt.xlabel('Epoch')
plt.ylabel('Clasification accuracy')
plt.show()

In [ ]:
from cs231n.vis_utils import visualize_grid

# 네트워크 가중치 시각화

def show_net_weights(net):
  W1 = net.params['W1']
  W1 = W1.reshape(32, 32, 3, -1).transpose(3, 0, 1, 2)
  plt.imshow(visualize_grid(W1, padding=3).astype('uint8'))
  plt.gca().axis('off')
  plt.show()

show_net_weights(net)

 # hyperparameters 튜닝하기

**무엇이 문제인가?**. 위의 시각화를 살펴보면, 손실이 더(혹은 덜) 선형적으로 감소하고 있음을 확인할 수 있습니다. 이것은 학습률이 너무 낮을 수 있음을 시사하는것 처럼 보입니다. 게다가 학습과 검증 정확도 사이에 차이가 없다는것은 우리가 사용한 모델이 적은 용량을 가지고 있음을 나타내고 용량을 증가시켜야될 필요가 있습니다. 반면에, 매우 큰 모델을 사용한다면 overfitting이 발생할 수 있는데, 이 경우 학습 정확도와 검증 정확도 사이에 매우 큰 차이가 나는 것을 확인할 수 있을 것입니다.

**튜닝**. hyperparameters를 튜닝하고 최종 성능에 어떻게 영향을 끼치는지에 대한 직관을 얻기위해 많은 연습을 해야합니다. 아래에선 다양한 hidden layer 크기, 학습률, numer of training epochs 와 정규화 강도를 포함한 hyperparameters의 값들로 실험해야 합니다. 또한 학습률의 decay를 튜닝하는 것에 대해 생각해 볼 수 있지만 아마 기본 값이 가장 좋은 성능을 낼 것입니다.

**결과 예측하기**. 검증 셋에 대한 분류 정확도를 48% 이상으로 만들 수 있게 목표삼도록 합시다. 우리의 가장 좋은 네트워크는 검증 셋에 대해 52%이상의 정확도를 얻었습니다.

**실험**: 이 연습에서 목표는 완전히 연결된 신경망으로 CIFAR-10에 좋은 결과를 얻는 것입니다. 테스트 세트에 대해 52%이상의 정확도에 대해 각 1%마다 추가 보너스 점수를 얻을 수 있습니다. 자신만의 기술에 대해서 적어넣으세요. (예. PCA로 차원 줄이기, dropout 추가하기, solver에 특징 추가하기 등).

In [ ]:
best_net = None # 여기에 가장 좋은 모델을 저장하세요.

#################################################################################
# TODO: 검증 셋을 이용하여 hyperparameters를 튜닝하세요. 가장 잘 학습된 모델은 best_net에 저장 #
# 하세요.                                                                         #
#                                                                               #
# 우리가 위에서 사용한 시각화를 비슷하게 사용하면 디버그하는데 도움이 될 것입니다;                  #
# 시각화를 통해 위에서 잘 학습되지 않은 네트워크와 중요한 질적 차이를 보일 것 입니다.               #
#                                                                               #
# 손으로 hyperparameters를 미세조정하는것은 재밌을 수 있지만, 이전 연습에서 했던 것 처럼 자동으로  #
# 가능한 hyperparameters를 찾는 코드를 작성하는 것이 더 유용하다는것을 알게 될 것입니다.         #
#################################################################################
pass
#################################################################################
#                                    코드의 끝                                    #
#################################################################################

In [ ]:
# 가장 좋은 네트워크의 가중치를 시각화 합니다.
show_net_weights(best_net)

# 테스트 세트로 실행하기
실험이 끝났다면, 최종으로 학습된 네트워크를 테스트 세트로 실행해 봅니다; 48%이상의 정확도를 얻어야 합니다.

**52%이상의 각 1% 마다 추가 점수를 얻을 수 있습니다.**

In [ ]:
test_acc = (best_net.predict(X_test) == y_test).mean()
print 'Test accuracy: ', test_acc